In [2]:
import zmq
import time
import socket
import os
import sys
import logging
import threading
import numpy as np
from numpy import matlib
from __future__ import division

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
print 'Computer: ' + comp_name

if 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    rec_folder = os.path.join('/usr/local/experiment/raw_data')
    

if 'lookfar' in comp_name:
    repos_folder = os.path.abspath('/Users/zeke/repos')
    experiment_folder = os.path.join('/Users/zeke/bci_zf')
    rec_folder = os.path.join('/Users/zeke/bci_zf/raw_data')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))

from file_tools import experiment as et
from rig_tools import open_ephys as oe, beagle_bone as bb

Computer: lookfar


In [3]:
# open the devices
oe_zmq = oe.OpenEphysEvents(port='5556', ip='127.0.0.1')
bb_zmq = bb.BeagleBone(port='5559', ip='tsippor.ucsd.edu')

In [4]:
oe_zmq.connect()
oe_zmq.stop_acq()
oe_zmq.start_acq()

No acquisition running
Acquisition Started


In [7]:
bb_zmq.connect()

In [8]:
# Bird specific data
bird_id = 'z033'
electrode_depth = 0 #append to rec folder
experiment_descr = None #prepend to rec folder

wave_files = [os.path.abspath('/root/experiment/stim/bos_tag.wav'),
              os.path.abspath('/root/experiment/stim/bos_rev_tag.wav'),
              os.path.abspath('/root/experiment/stim/con_tag.wav')]

# wave_files = [os.path.abspath('/root/experiment/stim/bos_tag.wav'), 
#               os.path.abspath('/root/experiment/stim/bos_rev_tag.wav')]
wave_file = wave_files[0]
cmd_line = 'trial type passive number 1 stim_file ' + wave_file
bb_zmq.send_command(cmd_line)

'ok trial:1, file:/root/experiment/stim/bos_tag.wav'

In [11]:
cmd_line = 'trial type passive number 1 stim_file ' + wave_files[0]

for i in range(5):
    bb_zmq.send_command(cmd_line)
    time.sleep(2)

In [7]:
# command functions
def cmd_play_wav(wav_file):
    cmd_line = 'play_wav stim_file ' + wav_file
    return cmd_line

def cmd_trial_pin(on):
    #on is boolean
    return 'trial_pin on {}'.format(on)

def cmd_trial_number(tr_num):
    #on is boolean
    return 'trial_number number {}'.format(int(tr_num))
        
def run_trial(rig_par={'bb': bb_zmq, 'oe':oe_zmq}, trial_par={'iti':500., 'tr_num': 5, 'wave_file': wave_file}):
    time.sleep(int(trial_par['iti']*100)/100000.)
    rig_par['bb'].send_command(cmd_trial_pin(True))
    rig_par['oe'].send_command(cmd_trial_number(trial_par['tr_num']))
    rig_par['bb'].send_command(cmd_trial_number(trial_par['tr_num']))
    rig_par['oe'].send_command(cmd_play_wav(trial_par['stim']))
    rig_par['bb'].send_command(cmd_play_wav(trial_par['stim']))
    rig_par['bb'].send_command(cmd_trial_pin(False))

bb_zmq.send_command(cmd_trial_pin(True))
bb_zmq.send_command(cmd_trial_number(4))
bb_zmq.send_command(cmd_play_wav(wave_file))
bb_zmq.send_command(cmd_trial_pin(False))
    
run_trial(rig_par={'bb': bb_zmq, 'oe': oe_zmq}, trial_par={'iti': 2000, 'tr_num': 15, 'stim': wave_file})

In [10]:
class Block:
    def __init__(self, stim_list, iti_bounds_ms, size, random = True):
        self.trials = 0
        self.iti_bounds = iti_bounds_ms
        self.stim_list = stim_list
        self.size = size
        self.trial_pars = {'iti': None, 'tr_num': 0, 'stim': None}
        self.stim_order = matlib.repmat(np.arange(len(stim_list)),
                                           int(np.ceil(size/len(stim_list))),
                                           1)[:size].flatten()
        if random:
            np.random.shuffle(self.stim_order)
    
    def __iter__(self,):
        return self
    
    def next(self, ):
        if self.trials < self.size:
            self.trial_pars['iti'] = np.random.randint(self.iti_bounds[0], self.iti_bounds[1])
            #print self.trials
            self.trial_pars['tr_num'] = self.trials
            self.trial_pars['stim'] = self.stim_list[self.stim_order[self.trials]]
            self.trials+=1
            return self.trial_pars
        else:
            raise StopIteration()
            
class Runner(threading.Thread):
    def __init__(self, block, rig_par, recorder=None, group=None, target=None, name=None, 
                 args=(), kwargs=None, verbose=None):
        threading.Thread.__init__(self, group=group, target=target, name=name, verbose=verbose)
        
        logging.info('Initializing block')
        self.block = block #block object
        self.rig = rig_par
        self.stim_sys = rig_par['bb']
        self.rec_sys = rig_par['oe']
        self.running = threading.Event() #Event
        self.finished = False
        self.trial = None
        self.running.clear()
        self.recorder = recorder #Record control
   
    def run(self):
        logging.debug('running block')
        if not self.running.is_set():
            print 'Starting Block'
#             if self.is_recording()==True:
#                 logging.debug('Starting recording')
#                 self.recorder.start()
#                 time.sleep(2)
            self.running.set()
            while True:
                try:
                    if self.running.is_set():
                        self.trial = self.block.next()
                        logging.info('Running trial {}'.format(self.trial))
                        print self.trial
                        run_trial(rig_par=self.rig, trial_par=self.trial)
                    else:
                        self.end_block()
                        break

                except StopIteration:
                    self.finished = True
                    self.running.clear()
                    print 'finished block'
                    break
            
            if self.is_recording()==True:
                print('Runner stopping recording')
                self.recorder.signal_stop()
        else:
            logging.info('Cant start, already running')
            
        return self.finished
                    
    def end_block(self):
        print('stopping the block in trial {}'.format(self.trial['tr_num']))
        self.finished = False
        
    def signal_stop(self):
        self.running.clear()
        
    def is_running(self):
        return self.running.is_set()
    
    def is_recording(self):
        logging.debug('runner checking recording status')
        logging.debug('{}'.format(self.recorder))
        if self.recorder is not None:
            logging.debug('status {}'.format(self.recorder.report_recording()))
            return self.recorder.report_recording()
        else:
            return None

class H5er():
    # To do: a thread to log the input/output of every trial, 
    # that is init by a session and passed to a runner
    def __init__(self):
        raise NotImplementedError
        
class Recorder(threading.Thread):
    def __init__(self, rig_par, control_Event, new=False, rec_par={'CreateNewDir': '0'}, restart=True, 
                 group=None, target=None, name=None, 
                 args=(), kwargs=None, verbose=None):
        threading.Thread.__init__(self, group=group, target=target, name=name, verbose=verbose)
        
        logging.info('Initializing recorder')
        self.name = 'Recorder'
        self.rig = rig_par
        self.rec_sys = rig_par['rec']
        self.recording = threading.Event() #Event
        self.control_signal = control_Event # Event
        self.recording_path = None
        self.rec_par = rec_par
        self.new_rec = new
        self.force_restart = restart
        self.recording.clear()
        
   
    def run(self):
        logging.debug('recording')
        # If starting of a new recorded is forced
        ok_to_start = False
        
        if self.is_recording():
            if self.force_restart:
                self.stop_record()
                ok_to_start = True
                
        if self.control_signal is not None:
            while not self.control_signal.is_set():
                pass
            logging.debug('Start record signal received')
            self.start_record(new=self.new_rec, rec_par=self.rec_par)
            while self.control_signal.is_set():
                if self.recording.is_set():
                    pass
                else:
                    logging.info('Stop recording Forced')
                    self.control_signal.clear()
                    break
            logging.debug('Stop record signal received')
            self.stop_record()

    def start_record(self, new=False, rec_par={'CreateNewDir':'0'}):
        if self.rec_sys.start_rec(rec_par=rec_par):
            logging.debug('Started recording')
            self.recording_path = self.rec_sys.get_rec_path()
            self.recording.set()
        return self.recording_path
                    
    def stop_record(self, ):
        if self.rec_sys.query_status('Recording'):
            self.rec_sys.stop_rec()
            self.recording.clear()
            logging.info('Stopped recording')
            
    def signal_stop(self):
        self.recording.clear()
        
    def is_recording(self):
        return self.rec_sys.query_status(status_query='Recording')
    
    def report_recording(self):
        return self.recording.is_set()
    
    def report_recording_parh(self):
        return self.recording_path()
    
    def get_rec_path(self):
        return self.rec_sys.get_rec_path()
    
    
bl = Block(wave_files, [300, 800], 5)
bl_rnr = Runner(bl, {'bb': bb_zmq, 'oe':oe_zmq})
bl_rnr.start()

Starting Block
{'tr_num': 0, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 613}
{'tr_num': 1, 'stim': '/root/experiment/stim/bos_syn_tag.wav', 'iti': 760}
{'tr_num': 2, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 651}
{'tr_num': 3, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 430}
{'tr_num': 4, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 741}
finished block


In [9]:
bl_rnr.signal_stop()

In [10]:
class Session:
    def __init__(self, ephys, beagle, bird_id='z000', depth=100, descr=None):
        self.ephys = ephys
        self.beagle = beagle
        self.bird_folder = None
        self.sess_folder = None
        self.bird_id = bird_id
        self.electrode_depth = depth #append to rec folder
        self.experiment_descr = descr #prepend to rec folder
        self.block_runner = None
        self.trial_number = 1
        self.make_bird_folder()
        self.recording = threading.Event()
        self.record_control = threading.Event()
        self.recorder = Recorder({'rec': self.ephys}, self.record_control)
        self.recording.clear()
        self.record_control.clear()
        
    def make_bird_folder(self, ):
        self.bird_folder = os.path.join(rec_folder, self.bird_id)
        et.mkdir_p(self.bird_folder)
    
    def start_record(self, new=False, force_restart=False):
        if new:
            rec_par = {'CreateNewDir': '1', 
                      'RecDir': self.bird_folder,
                      'AppendText': str(self.electrode_depth),
                      'PrependText': self.experiment_descr}
        else:
            rec_par = {'CreateNewDir':'0'}
        
        if self.is_recording():
            logging.info('Trying to record but already recording')
        
        else:
            self.record_control.set()
            self.recorder = Recorder({'rec': self.ephys}, self.record_control,
                                    new=new, rec_par=rec_par, restart=force_restart)
            self.recorder.start()
            if self.recorder.report_recording():
                logging.debug('Started recording')
                self.sess_folder = self.recorder.report_recording_path()
                self.recording.set()
    
    def stop_record(self, ):
        if self.recorder.report_recording():
            self.recorder.signal_stop()
            self.recording.clear()
            logging.info('Stopped recording')
            
    def start_block(self, block, record=False, new=False):
        if record:
            logging.info('Starting to record')
            self.start_record(new)
            time.sleep(2)
            
        logging.info('Starting block')
        self.block_runner = Runner(block, {'bb': self.beagle, 'oe': self.ephys}, 
                                   recorder=self.recorder)
        self.block_runner.start()
        
    def monitor_block(self, ):
        if self.is_recording():
            self.stop_record()
    
    def stop_block(self):
        self.block_runner.signal_stop()
        if self.is_recording:
            self.stop_record()
        
    def is_recording(self):
        return self.recording.is_set()
    
    def is_running(self):
        return self.block_runner.is_running()
    
    
ss = Session(oe_zmq, bb_zmq, bird_id='z007', depth=100, descr='test')

In [18]:
wave_files

Recording stopped


['/root/experiment/stim/bos01_tag.wav',
 '/root/experiment/stim/bos02_tag.wav',
 '/root/experiment/stim/bos03_tag.wav',
 '/root/experiment/stim/bos01-rev_tag.wav',
 '/root/experiment/stim/con01_tag.wav']

In [23]:
waves_to_use = [wave_files[0], wave_files[3]]

In [24]:
waves_to_use

['/root/experiment/stim/bos01_tag.wav',
 '/root/experiment/stim/bos01-rev_tag.wav']

In [25]:
# run the one session
ss = Session(oe_zmq, bb_zmq, bird_id='b1114', depth=1, descr='anesth_surface')
a_block = Block(waves_to_use, [5000, 12000], 150)
ss.start_block(a_block, record=True, new=True)
#time.sleep(5)
#ss.stop_block()

Was not Acquiring
Acquisition Started
OK to start
Recording path: /Users/zeke/bci_zf/raw_data/b1114/anesth_surface_2017-03-10_16-30-53_1
Starting Block
{'tr_num': 0, 'stim': '/root/experiment/stim/bos01-rev_tag.wav', 'iti': 7409}


In [22]:
ss.stop_block()

Recording stopped
stopping the block in trial 92


In [65]:
# run a night experiment
block_trials = 150
block_iti = [2000, 7000]
time.sleep(10)
ss = Session(oe_zmq, bb_zmq, bird_id='z023', depth=0, descr='anesth_surf')
blocks = []
for i_block in range(1):
    print "Starting block {}".format(i_block)
    a_block = Block(wave_files, block_iti, block_trials)
    ss.start_block(a_block, record=True, new=True)
    #time.sleep(block_trials*int(np.ceil(block_iti[1]*0.0011)))
    time.sleep(1600)
    ss.stop_block()
    print 'Block {} finished'.format(i_block)
    blocks.append(a_block)
    time.sleep(1)
    

Starting block 0
OK to start
Recording path: /Users/zeke/bci_zf/raw_data/z009/anesth_surface_2016-11-04_16-19-47_0
Starting Block
{'tr_num': 0, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 3815}
{'tr_num': 1, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 38226}
{'tr_num': 2, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 11399}
{'tr_num': 3, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 65405}
{'tr_num': 4, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 18873}
{'tr_num': 5, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 38545}
{'tr_num': 6, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 20627}
{'tr_num': 7, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 55375}
{'tr_num': 8, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 51975}
{'tr_num': 9, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 40017}
{'tr_num': 10, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 7983}
{'tr_num': 11, 'stim': '/root/experiment/stim/bos_ta

In [41]:
sys.stdout.flush()


In [19]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [76]:
for i in range(1):
    print i

0


In [77]:
150*10

1500

In [ ]:
rec_opt = ['{0}={1}'.format(key, value)
                       for key, value in rec_par.iteritems()
                       if value is not None]